## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-16-16-51-25 +0000,nyt,Where Does the Israel-Hamas Cease-Fire Stand?,https://www.nytimes.com/2025/10/16/world/middl...
1,2025-10-16-16-49-40 +0000,nyt,White House Set to Announce Proposal Seeking t...,https://www.nytimes.com/2025/10/16/us/politics...
2,2025-10-16-16-49-09 +0000,nypost,Boater bros trash Playland amusement park on r...,https://nypost.com/2025/10/16/us-news/boater-b...
3,2025-10-16-16-49-04 +0000,nypost,"Sen. Mitch McConnell, 83, falls in Senate offi...",https://nypost.com/2025/10/16/us-news/sen-mitc...
4,2025-10-16-16-49-00 +0000,wsj,Ray-Ban Maker EssilorLuxottica’s Revenue Surge...,https://www.wsj.com/business/retail/ray-ban-ma...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,59
232,new,27
186,china,19
84,will,18
213,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
246,2025-10-16-02-09-00 +0000,wsj,Democrats and Republicans both feel they have ...,https://www.wsj.com/politics/policy/government...,136
46,2025-10-16-15-47-55 +0000,cbc,"Trump's Gaza deal may be 'historic,' but falls...",https://www.cbc.ca/news/world/gaza-trump-peace...,125
80,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...,122
344,2025-10-15-19-58-31 +0000,latimes,California judge halts Trump federal job cuts ...,https://www.latimes.com/politics/story/2025-10...,116
89,2025-10-16-14-19-27 +0000,bbc,India casts doubt on Trump's claim Modi will s...,https://www.bbc.com/news/articles/c8eykdy0567o...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
246,136,2025-10-16-02-09-00 +0000,wsj,Democrats and Republicans both feel they have ...,https://www.wsj.com/politics/policy/government...
80,63,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...
334,57,2025-10-15-20-20-41 +0000,nypost,Bodies of 2 more hostages returned to Israel —...,https://nypost.com/2025/10/15/world-news/red-c...
221,52,2025-10-16-07-13-35 +0000,nypost,Just-released hostage attends funeral of fello...,https://nypost.com/2025/10/16/world-news/just-...
173,45,2025-10-16-10-12-08 +0000,nyt,China Fans Patriotic Sentiment as Trade War Wi...,https://www.nytimes.com/2025/10/16/world/asia/...
98,42,2025-10-16-14-00-14 +0000,nypost,"Chaos erupts after Kenyan forces fire in air, ...",https://nypost.com/2025/10/16/world-news/kenya...
95,39,2025-10-16-14-02-46 +0000,wapo,Do you need a better job? Answer these five qu...,https://www.washingtonpost.com/business/2025/1...
358,38,2025-10-15-19-08-54 +0000,bbc,Syria's Sharaa meets Putin in Moscow for first...,https://www.bbc.com/news/articles/c201p2dd6r4o...
255,38,2025-10-16-01-36-00 +0000,wsj,More than three dozen organizations and indivi...,https://www.wsj.com/politics/policy/trump-ball...
90,35,2025-10-16-14-12-57 +0000,nypost,Goldman’s No. 2 exec urges end to US shutdown:...,https://nypost.com/2025/10/16/business/goldman...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
